here we discuss a lot of the essential functionality common to the pandas data structure. Here's the some of the object used in the examples from previous section

In [4]:
import pandas as pd
import numpy as np

In [5]:
index = pd.date_range('1/1/2000', periods=8)
s = pd.Series(np.random.randn(5), index=['a','b','c','d','e'])
df = pd.DataFrame(np.random.randn(8,3), index=index, columns=['A','B','C',])
wp = pd.Panel(np.random.randn(2, 5, 4), items=['item1', 'item2'],
                  major_axis=pd.date_range('1/1/2000', periods=5),
                  minor_axis=['A','B','C','D']
              )

### Head and Tail

To view a small sample of a Sereis or DataFrame, use the head() and tail() methods. The defalut number of elements to display is five, but you may pass a custom number.

In [6]:
long_series = pd.Series(np.random.randn(1000))
print long_series.head()
print long_series.tail(3)

SyntaxError: invalid syntax (<ipython-input-6-dab0ba47444e>, line 2)

### Attributes and the raw ndarray(s)

pandas object have a number of attributes enabling you to aceess the metadata

- shape: give the axis dimetion of the object. consistent with ndarray
- Axis labels
> - Sereis: index (only axis)
  - DataFrame : index(row) and columns
  - Panel : itmes. major_axis, and minor_axis
  
Note, there attributes can be safely assigned to

In [ ]:
print df[:2]
print [x for x in df.columns]
df.columns = [x.lower() for x in df.columns]
print df

To get the actual data inside a data structure, one need only acesss the values property

In [ ]:
print s.values
print df.values
print wp.values

if a DataFrame or Panel homogeneously-typed data, the ndarray can actually be modified in place, and the changes will be reflected in the data structure. For heterogeneous data (e.g some of the DataFrame's columns are not all the same dtype), this will not be case. the values attribute itself, unlike the axis labels, cannot be assigned to.

> *Note* When working with hetergeneous data, the dtype of the resulting ndarray will be choosen to accommodate all of the data involved.
For example. if string are involved, the result will be of object dtype. if there are only float and integers, the resulting array will be of float dtype

## Flexible binary operations
with binary operation between pandas data structures, there are two key point of interest:

- broadcasting behavior between higher ( e.g DataFrame) and lower-dimensional(e.g Series) objects
- missing data in computation



### Matching / broadcasting behavior

DataFrame has the methods add, sub, mul, div and related function radd, rsub... for carrying out binary operations. For broadcasting behavior. Series input is of primary interest, Using these functions, you can use to either match on the index or columns via the axis keyword

In [ ]:
df = pd.DataFrame({
        'one': pd.Series(np.random.randn(3), index=['a', 'b', 'c']),
        'two': pd.Series(np.random.randn(4), index=['a', 'b', 'c', 'd']),
        'three': pd.Series(np.random.randn(3), index=['b', 'c', 'd'])
    })

print df
row = df.ix[1]
print row
column = df['two']
print column

print df.sub(row, axis='columns')
print df.sub(row, axis=1)
print df.sub(column, axis='index')
print df.sub(column, axis=0)


Futuremore you can align a level of a multi-indexed DataFrame with a Series

In [ ]:
dfmi = df.copy()
dfmi.indes = pd.MultiIndex.from_tuples([(1, 'a'), (1, 'b'), (1, 'c'),(2, 'a')], names=['first','second'])
print dfmi.sub(column, axis=0, level='second')
print dfmi

### Missing data / operation with fill values

in Series and DataFrame (though not yet in Panel), the arithmetic funcitons have the option of inputting a fill_values, namely a value to substitute when at most one of the values at a location are missing.
For Example when adding two dataFrame objects,  you may wish to treat NaN as 0 Unless both DataFrame are missing that values, in which case the result will be NaN( you can later replace NaN with some other values using fillna if you wish)

In [ ]:
print df
df2 = df.copy()
df2.iloc[0][1] = 12
print df2
print df + df2
print df.add(df2, fill_value=0)

### Flexible Comparsions

pandas introduced binary comparsion methods eq, ne, lt, gt, le and ge to Series and DataFrame whose behavior is analogous to the binary artimetic operations described above:

In [ ]:
print df.gt(df2)
print df.ne(df2)

### Boolean Redcutions

you can apply the reductions: empty, any, all and boolean to provide a way to summarize a boolean result

In [ ]:
print df > 0
print (df > 0).all()
print (df > 0).all(axis='columns')
print (df > 0).any()

### Descriptive statistic

A large number of methods for computing descriptive statistics and other related operation on Series, DataFrame, Panel. Most of these are aggregation(hence producing a lower-dimensional result) like sum(), mean(), and quantile(), but more of them, like cumsum(), cumprod(), produce an object of the same size. Generally speaking there methods take an axis argument, just like ndarray {sum, std,...} but the axis can be specified by name or integer

- Sereis: no axis argument needed
- DataFrame: "index"(axis=0, default), "column"(axis=1)
- Panel: "item"(axis=0),"major"(axis=1,default), "minor"(axis=2)

For example:

In [ ]:
print df
print df.mean()
print df.mean(1)

All such methods have a skipna option signaling whether to exclude missing data(True by default)

In [ ]:
print df.sum(0, skipna=False)
print df.sum(1, skipna=True)

combined with the broadcasting / aritmetic behavior, one can descibe various statistical procedure, like standardization (rendering data zero mean and standard deviation 1), very concisely

In [ ]:
ts_stand = (df - df.mean())/ df.std()
print ts_stand
print ts_stand.std()

xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
print xs_stand.std(1)

Note that method like cumsum() and cumprod() preserve the location of NA values:

In [ ]:
print df
print df.cumsum()

here is a quick reference sumarry table of common functions. Each also takes an optional level parameter which applies only if the object has hierarchical index
- Function : Description
- count: Number of non-null observations
- sum: sum of values
- mean: mean of values
- mad: mean absolute deviation
- median: artimetic median of values
- min : minimun
- max: maximun
- mode: xmode
- abs: absoulte value
- prod: product of values
- std: Bessel-corrected sample standart defiveaton
- var: unvised variance
- sem: stadard error of the mean
- skew: sample skwe
- kurt: Sample kurtoris
- quantile: smaple quantile
- comsum: cumulative sum
- cumprod: 